In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# ==============================================================================
#  环境设置与依赖检查
# ==============================================================================

# 设置 Keras 后端为 PyTorch，确保在导入 Keras 之前执行
os.environ['KERAS_BACKEND'] = 'torch'

try:
    import keras
    import torch # 导入 torch 以进行设备检查
    from packaging.version import parse as parse_version
    
    # 【核心】增加 Keras 版本检查
    if parse_version(keras.__version__) < parse_version("3.0.0"):
        print("="*80)
        print("错误：检测到过时的 Keras 版本。")
        print(f"您当前安装的 Keras 版本是 {keras.__version__}，但 TKAN 库需要 Keras 3 或更高版本。")
        print("Keras 3 引入了 'keras.ops' 模块，这是 TKAN 正常运行所必需的。")
        print("\n请在您的 conda 环境中运行以下命令来升级：")
        print("pip install --upgrade keras tensorflow")
        print("="*80)
        exit() # 终止程序

    # 【核心修正】修正 TKAN 的导入路径
    from tkan import TKAN 

except ImportError as e:
    # 捕获其他导入错误，并提供清晰的指导
    if "No module named 'tensorflow'" in str(e):
        print("="*80)
        print("错误：缺少 TensorFlow 依赖。")
        print("Keras 3 框架，即使在 PyTorch 后端模式下，也需要安装 TensorFlow 才能正常导入。")
        print("\n请在您的 conda 环境中运行以下命令来安装：")
        print("pip install tensorflow")
        print("="*80)
        exit() # 终止程序
    elif "No module named 'torch'" in str(e):
        print("="*80)
        print("错误：缺少 PyTorch 依赖。")
        print("您已将 Keras 后端设置为 'torch'，但当前环境中未找到 PyTorch。")
        print("\n请在您的 conda 环境中运行以下命令来安装：")
        print("pip install torch torchvision torchaudio")
        print("="*80)
        exit()
    else:
        # 如果是其他导入错误，则正常抛出
        raise e

# ==============================================================================
#  【核心新增】设备验证模块
# ==============================================================================
print("\n" + "="*80)
print("--- 步骤 0: 设备验证 ---")
try:
    if torch.cuda.is_available():
        DEVICE_NAME = "cuda"
        print("✅ PyTorch 检测到 CUDA GPU 可用。")
        print(f"  - GPU 型号: {torch.cuda.get_device_name(0)}")
    else:
        DEVICE_NAME = "cpu"
        print("❌ 警告：PyTorch 未能找到可用的 CUDA GPU。")

    # Keras 3 (torch backend) 会遵循 torch 的设备
    # 我们可以通过创建一个张量来验证 Keras 当前的默认设备
    test_tensor = keras.ops.convert_to_tensor([1.0])
    print(f"✅ Keras 张量默认被创建在: {test_tensor.device}")
    
    if DEVICE_NAME == "cpu" and "cpu" not in str(test_tensor.device):
        print("⚠️ 注意：PyTorch 未找到 GPU，但 Keras 似乎指向了非 CPU 设备。可能存在配置冲突。")
    elif DEVICE_NAME == "cuda" and "cuda" not in str(test_tensor.device):
        print("❌ 错误：PyTorch 找到了 GPU，但 Keras 未能使用它！训练将在 CPU 上进行。")
    else:
        print("✅ Keras 与 PyTorch 设备匹配。")

except Exception as e:
    DEVICE_NAME = "cpu"
    print(f"❌ 设备验证失败: {e}。将默认使用 CPU。")
print(f"结论：模型将运行在 '{DEVICE_NAME.upper()}' 设备上。")
print("="*80)
# ==============================================================================


# ==============================================================================
#  第一部分：数据加载与预处理 (来自您的代码)
# ==============================================================================
print("\n--- 步骤 1: 加载与预处理数据 ---")
dictory = "../Datasets/"
filename = "0.275_Speed_OB.csv"
data = pd.read_csv(dictory+filename)
# data = pd.read_csv(r'../Datasets/0.15_Speed_withouOB.csv')
print(f"成功加载数据集，总行数: {len(data)}")

# 定义分割比例
train_ratio = 0.8
val_ratio = 0.1
data_size = len(data)
train_end_idx = int(train_ratio * data_size)
val_end_idx = train_end_idx + int(val_ratio * data_size)

# 使用 .iloc 分割 DataFrame
train_data = data.iloc[:train_end_idx]
val_data = data.iloc[train_end_idx:val_end_idx]
test_data = data.iloc[val_end_idx:]

# train_data = pd.read_csv(
#     r'../Datasets/zoulang/train.csv')
# val_data = pd.read_csv(
#     r'../Datasets/zoulang/validation.csv')
# test_data = pd.read_csv(
#     r'../Datasets/zoulang/test.csv')

print("\nData loaded successfully!")
print(f"Training data shape:   {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Testing data shape:    {test_data.shape}")

column_names = train_data.drop(['timestamp','x_coord', 'y_coord'], axis=1).columns.tolist()
wifi_features = [col for col in column_names if any(sensor in col for sensor in ["rot", "RSSI"])]
# wifi_features = [col for col in column_names if any(sensor in col for sensor in ["RSSI","distance"])] 
# imu_features = [col for col in column_names if any(sensor in col for sensor in ["accelerometer", "gyroscope"])]
imu_features = [col for col in column_names if any(sensor in col for sensor in ["accelerometer", "gyroscope","magnetometer"])] 
print(f"\nIdentified {len(wifi_features)} WiFi features and {len(imu_features)} IMU features.")
coord_cols = ['x_coord', 'y_coord']
# 在DataFrame上进行标准化
scaler_wifi = StandardScaler().fit(train_data[wifi_features])
scaler_imu = StandardScaler().fit(train_data[imu_features])
coords=train_data[['x_coord','y_coord']]
scaler = StandardScaler()
scaler.fit(coords)

train_df_scaled = train_data.copy()
val_df_scaled = val_data.copy()
test_df_scaled = test_data.copy()

for df in [train_df_scaled, val_df_scaled, test_df_scaled]:
    df[wifi_features] = scaler_wifi.transform(df[wifi_features])
    df[imu_features] = scaler_imu.transform(df[imu_features])
    df[coord_cols] = scaler.transform(df[coord_cols])

print("数据已在DataFrame上完成标准化。")

# ==============================================================================
#  第二部分：数据塑形 - 创建滑动窗口
# ==============================================================================
def create_sliding_windows(df, feature_cols, label_cols, window_size, future_steps):
    """
    将时序 DataFrame 转换为适用于 Keras RNN 模型的滑动窗口数据。
    """
    X, y = [], []
    num_samples = len(df) - window_size - future_steps + 1
    
    feature_data = df[feature_cols].values
    label_data = df[label_cols].values
    
    for i in range(num_samples):
        feature_window = feature_data[i : i + window_size]
        X.append(feature_window)
        label_window = label_data[i + window_size : i + window_size + future_steps]
        y.append(label_window.flatten())
        
    return np.array(X), np.array(y)

print("\n--- 步骤 2: 创建滑动窗口数据 ---")
windows_size = 30
future_steps = 3

all_features = wifi_features + imu_features
label_cols = ['x_coord', 'y_coord']

X_train, y_train = create_sliding_windows(train_df_scaled, all_features, label_cols, windows_size, future_steps)
X_val, y_val = create_sliding_windows(val_df_scaled, all_features, label_cols, windows_size, future_steps)
X_test, y_test = create_sliding_windows(test_df_scaled, all_features, label_cols, windows_size, future_steps)

print("滑动窗口数据创建完毕！")
print(f"  - X_train shape: {X_train.shape}")
print(f"  - y_train shape: {y_train.shape}")

# ==============================================================================
#  第三部分：Keras TKAN 模型训练与评估
# ==============================================================================
print("\n--- 步骤 3: 构建与训练 Keras TKAN 模型 ---")

input_shape = (X_train.shape[1], X_train.shape[2])
output_dim = y_train.shape[1]

# 【注意】Keras with PyTorch backend 会自动将模型和数据移动到检测到的设备
inputs = keras.Input(shape=input_shape)
tkan_output = TKAN(units=32, return_sequences=False, name="TKAN_Layer")(inputs)
outputs = keras.layers.Dense(output_dim, name="Prediction_Head")(tkan_output)
model = keras.Model(inputs, outputs)
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.Huber(),
    metrics=['mse']
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# 【注释】这是真正的“训练”步骤，模型的权重会在这里被更新。
print("\n--- 开始模型训练 (model.fit) ---")
history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=2
)
print("--- 模型训练结束 ---")


# ==============================================================================
#  第四部分：评估
# ==============================================================================
# 【注释】下面的步骤是在训练结束后，使用已经训练好的最佳模型在“测试集”上进行评估。
# 【注释】模型的权重在此阶段是冻结的，不会再改变。
print("\n--- 步骤 4: 在测试集上评估最终模型 ---")

# 【注释】model.evaluate()：计算测试集上的损失。您会看到一个短暂的进度条。
print("\n--- 开始在测试集上评估 (model.evaluate) ---")
test_loss, test_mse = model.evaluate(X_test, y_test, verbose=1)

# 【注释】model.predict()：生成对测试集的具体预测值，用于计算RMSE/ADE。您会看到另一个短暂的进度条。
print("\n--- 开始在测试集上预测 (model.predict) ---")
y_pred = model.predict(X_test)

# 【修正】修正了 y_test_seq 的 reshape 笔误
y_test_seq = y_test.reshape(-1, future_steps, 2)
y_pred_seq = y_pred.reshape(-1, future_steps, 2)

true_coords_scaled = y_test_seq[:, -1, :]
pred_coords_scaled = y_pred_seq[:, -1, :]

true_coords = np.concatenate([
    scaler_x.inverse_transform(true_coords_scaled[:, 0:1]),
    scaler_y.inverse_transform(true_coords_scaled[:, 1:2])
], axis=1)

pred_coords = np.concatenate([
    scaler_x.inverse_transform(pred_coords_scaled[:, 0:1]),
    scaler_y.inverse_transform(pred_coords_scaled[:, 1:2])
], axis=1)

rmse = np.sqrt(np.mean((true_coords - pred_coords)**2))
ade = np.mean(np.sqrt(np.sum((true_coords - pred_coords)**2, axis=1)))

print("\n--- Keras TKAN 模型评估结果 ---")
print(f"Test Loss (Huber): {test_loss:.4f}")
print(f"Test MSE: {test_mse:.4f} (米^2, 标准化后)")
print("-" * 20)
print(f"总均方根误差 (RMSE): {rmse:.4f} (米)")
print(f"平均位移误差 (ADE/FDE): {ade:.4f} (米)")
print("---------------------------------")


# ==============================================================================
#  第五部分：保存最终预测结果
# ==============================================================================
print("\n--- 步骤 5: 保存预测结果到 CSV 文件 ---")
results_df = pd.DataFrame({
    'True_X': true_coords[:, 0],
    'True_Y': true_coords[:, 1],
    'Pred_X': pred_coords[:, 0],
    'Pred_Y': pred_coords[:, 1]
})

output_filename = 'results/tkan_zoulang.csv'
os.makedirs('results', exist_ok=True) # 确保 results 文件夹存在
results_df.to_csv(output_filename, index=False)

print(f"✅ 预测结果已成功保存到: {output_filename}")



--- 步骤 0: 设备验证 ---
✅ PyTorch 检测到 CUDA GPU 可用。
  - GPU 型号: NVIDIA GeForce RTX 3070 Ti
✅ Keras 张量默认被创建在: cuda:0
✅ Keras 与 PyTorch 设备匹配。
结论：模型将运行在 'CUDA' 设备上。

--- 步骤 1: 加载与预处理数据 ---
成功加载数据集，总行数: 20302

Data loaded successfully!
Training data shape:   (16241, 22)
Validation data shape: (2030, 22)
Testing data shape:    (2031, 22)

Identified 13 WiFi features and 6 IMU features.
数据已在DataFrame上完成标准化。

--- 步骤 2: 创建滑动窗口数据 ---
滑动窗口数据创建完毕！
  - X_train shape: (16209, 30, 19)
  - y_train shape: (16209, 6)

--- 步骤 3: 构建与训练 Keras TKAN 模型 ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30, 19)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ TKAN_Layer (TKAN)               │ (None, 32)             │         9,166 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Prediction_Head (Dense)         │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,364 (36.58 KB)

 Trainable params: 9,174 (35.84 KB)

 Non-trainable params: 190 (760.00 B)


--- 开始模型训练 (model.fit) ---
Epoch 1/50
254/254 - 58s - 228ms/step - loss: 0.2655 - mse: 0.5626 - val_loss: 0.0684 - val_mse: 0.1371
Epoch 2/50
254/254 - 59s - 232ms/step - loss: 0.0412 - mse: 0.0825 - val_loss: 0.0382 - val_mse: 0.0764
Epoch 3/50
254/254 - 59s - 232ms/step - loss: 0.0284 - mse: 0.0568 - val_loss: 0.0343 - val_mse: 0.0686
Epoch 4/50
254/254 - 60s - 235ms/step - loss: 0.0213 - mse: 0.0426 - val_loss: 0.0318 - val_mse: 0.0635
Epoch 5/50
254/254 - 61s - 240ms/step - loss: 0.0154 - mse: 0.0308 - val_loss: 0.0266 - val_mse: 0.0533
Epoch 6/50
254/254 - 61s - 240ms/step - loss: 0.0109 - mse: 0.0218 - val_loss: 0.0177 - val_mse: 0.0355
Epoch 7/50
254/254 - 62s - 243ms/step - loss: 0.0078 - mse: 0.0156 - val_loss: 0.0161 - val_mse: 0.0321
Epoch 8/50
254/254 - 62s - 243ms/step - loss: 0.0057 - mse: 0.0115 - val_loss: 0.0138 - val_mse: 0.0275
Epoch 9/50
254/254 - 62s - 245ms/step - loss: 0.0044 - mse: 0.0088 - val_loss: 0.0121 - val_mse: 0.0241
Epoch 10/50
254/254 - 63s - 247ms/st

KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# ==============================================================================
#  环境设置与依赖检查
# ==============================================================================

# 设置 Keras 后端为 PyTorch，确保在导入 Keras 之前执行
os.environ['KERAS_BACKEND'] = 'torch'

try:
    import keras
    import torch # 导入 torch 以进行设备检查
    from packaging.version import parse as parse_version

    # 【核心】增加 Keras 版本检查
    if parse_version(keras.__version__) < parse_version("3.0.0"):
        print("="*80)
        print("错误：检测到过时的 Keras 版本。")
        print(f"您当前安装的 Keras 版本是 {keras.__version__}，但 TKAN 库需要 Keras 3 或更高版本。")
        print("Keras 3 引入了 'keras.ops' 模块，这是 TKAN 正常运行所必需的。")
        print("\n请在您的 conda 环境中运行以下命令来升级：")
        print("pip install --upgrade keras tensorflow")
        print("="*80)
        exit() # 终止程序

    # 【核心修正】修正 TKAN 的导入路径
    from tkan import TKAN

except ImportError as e:
    # 捕获其他导入错误，并提供清晰的指导
    if "No module named 'tensorflow'" in str(e):
        print("="*80)
        print("错误：缺少 TensorFlow 依赖。")
        print("Keras 3 框架，即使在 PyTorch 后端模式下，也需要安装 TensorFlow 才能正常导入。")
        print("\n请在您的 conda 环境中运行以下命令来安装：")
        print("pip install tensorflow")
        print("="*80)
        exit() # 终止程序
    elif "No module named 'torch'" in str(e):
        print("="*80)
        print("错误：缺少 PyTorch 依赖。")
        print("您已将 Keras 后端设置为 'torch'，但当前环境中未找到 PyTorch。")
        print("\n请在您的 conda 环境中运行以下命令来安装：")
        print("pip install torch torchvision torchaudio")
        print("="*80)
        exit()
    else:
        # 如果是其他导入错误，则正常抛出
        raise e

# ==============================================================================
#  【核心新增】设备验证模块
# ==============================================================================
print("\n" + "="*80)
print("--- 步骤 0: 设备验证 ---")
try:
    if torch.cuda.is_available():
        DEVICE_NAME = "cuda"
        print("✅ PyTorch 检测到 CUDA GPU 可用。")
        print(f"  - GPU 型号: {torch.cuda.get_device_name(0)}")
    else:
        DEVICE_NAME = "cpu"
        print("❌ 警告：PyTorch 未能找到可用的 CUDA GPU。")

    # Keras 3 (torch backend) 会遵循 torch 的设备
    # 我们可以通过创建一个张量来验证 Keras 当前的默认设备
    test_tensor = keras.ops.convert_to_tensor([1.0])
    print(f"✅ Keras 张量默认被创建在: {test_tensor.device}")

    if DEVICE_NAME == "cpu" and "cpu" not in str(test_tensor.device):
        print("⚠️ 注意：PyTorch 未找到 GPU，但 Keras 似乎指向了非 CPU 设备。可能存在配置冲突。")
    elif DEVICE_NAME == "cuda" and "cuda" not in str(test_tensor.device):
        print("❌ 错误：PyTorch 找到了 GPU，但 Keras 未能使用它！训练将在 CPU 上进行。")
    else:
        print("✅ Keras 与 PyTorch 设备匹配。")

except Exception as e:
    DEVICE_NAME = "cpu"
    print(f"❌ 设备验证失败: {e}。将默认使用 CPU。")
print(f"结论：模型将运行在 '{DEVICE_NAME.upper()}' 设备上。")
print("="*80)
# ==============================================================================


# ==============================================================================
#  第一部分：数据加载与预处理
# ==============================================================================
# print("\n--- 步骤 1: 加载与预处理数据 ---")
# train_data = pd.read_csv(
#     r'../Datasets/zoulang/train.csv')
# val_data = pd.read_csv(
#     r'../Datasets/zoulang/validation.csv')
# test_data = pd.read_csv(
#     r'../Datasets/zoulang/test.csv')




dictory = "../Datasets/"
filename = "0.15_Speed_withoutOB.csv"
data = pd.read_csv(dictory+filename)
# data = pd.read_csv(r'../Datasets/0.45_Speed_withouOB.csv')
print(f"成功加载数据集，总行数: {len(data)}")

# 定义分割比例
train_ratio = 0.8
val_ratio = 0.1
data_size = len(data)
train_end_idx = int(train_ratio * data_size)
val_end_idx = train_end_idx + int(val_ratio * data_size)

# 使用 .iloc 分割 DataFrame
train_data = data.iloc[:train_end_idx]
val_data = data.iloc[train_end_idx:val_end_idx]
test_data = data.iloc[val_end_idx:]

print("\nData loaded successfully!")
print(f"Training data shape:   {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Testing data shape:    {test_data.shape}")

column_names = train_data.drop(['timestamp','x_coord', 'y_coord'], axis=1).columns.tolist()
wifi_features = [col for col in column_names if any(sensor in col for sensor in ["RSSI","distance","rot"])]
imu_features = [col for col in column_names if any(sensor in col for sensor in ["accelerometer", "gyroscope","magnetometer"])]
print(f"\nIdentified {len(wifi_features)} WiFi features and {len(imu_features)} IMU features.")
coord_cols = ['x_coord', 'y_coord']

# --- 【核心逻辑】标准化 ---
# 1. 只在训练数据上 fit scaler
scaler_wifi = StandardScaler().fit(train_data[wifi_features])
scaler_imu = StandardScaler().fit(train_data[imu_features])
scaler_coords = StandardScaler().fit(train_data[coord_cols])

# 2. 对所有数据集进行 transform
train_df_scaled = train_data.copy()
val_df_scaled = val_data.copy()
test_df_scaled = test_data.copy()

for df in [train_df_scaled, val_df_scaled, test_df_scaled]:
    df[wifi_features] = scaler_wifi.transform(df[wifi_features])
    df[imu_features] = scaler_imu.transform(df[imu_features])
    df[coord_cols] = scaler_coords.transform(df[coord_cols])

print("数据已在DataFrame上完成标准化。")

# ==============================================================================
#  第二部分：数据塑形 - 创建滑动窗口
# ==============================================================================
def create_sliding_windows(df, feature_cols, label_cols, window_size, future_steps):
    X, y = [], []
    num_samples = len(df) - window_size - future_steps + 1
    
    feature_data = df[feature_cols].values
    label_data = df[label_cols].values
    
    for i in range(num_samples):
        feature_window = feature_data[i : i + window_size]
        X.append(feature_window)
        label_window = label_data[i + window_size : i + window_size + future_steps]
        y.append(label_window.flatten())
        
    return np.array(X), np.array(y)

print("\n--- 步骤 2: 创建滑动窗口数据 ---")
windows_size = 20
future_steps = 3

all_features = wifi_features + imu_features
label_cols = ['x_coord', 'y_coord']

X_train, y_train = create_sliding_windows(train_df_scaled, all_features, label_cols, windows_size, future_steps)
X_val, y_val = create_sliding_windows(val_df_scaled, all_features, label_cols, windows_size, future_steps)
X_test, y_test = create_sliding_windows(test_df_scaled, all_features, label_cols, windows_size, future_steps)

print("滑动窗口数据创建完毕！")
print(f"  - X_train shape: {X_train.shape}")
print(f"  - y_train shape: {y_train.shape}")

# ==============================================================================
#  第三部分：Keras TKAN 模型训练与评估
# ==============================================================================
print("\n--- 步骤 3: 构建与训练 Keras TKAN 模型 ---")

input_shape = (X_train.shape[1], X_train.shape[2])
output_dim = y_train.shape[1]

inputs = keras.Input(shape=input_shape)
tkan_output = TKAN(units=32, return_sequences=False, name="TKAN_Layer")(inputs)
outputs = keras.layers.Dense(output_dim, name="Prediction_Head")(tkan_output)
model = keras.Model(inputs, outputs)
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.Huber(),
    metrics=['mse']
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

print("\n--- 开始模型训练 (model.fit) ---")
history = model.fit(
    X_train,
    y_train,
    shuffle=False,
    batch_size=64,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)
print("--- 模型训练结束 ---")


# ==============================================================================
#  第四部分：评估
# ==============================================================================
print("\n--- 步骤 4: 在测试集上评估最终模型 ---")

print("\n--- 开始在测试集上评估 (model.evaluate) ---")
test_loss, test_mse = model.evaluate(X_test, y_test, verbose=1)

print("\n--- 开始在测试集上预测 (model.predict) ---")
y_pred = model.predict(X_test)

# --- 【核心修正】反标准化逻辑 ---
y_test_seq = y_test.reshape(-1, future_steps, 2)
y_pred_seq = y_pred.reshape(-1, future_steps, 2)

# 我们关心最后一步的预测
true_coords_scaled = y_test_seq[:, -1, :]
pred_coords_scaled = y_pred_seq[:, -1, :]

# 使用同一个 scaler_coords 进行反标准化
true_coords = scaler_coords.inverse_transform(true_coords_scaled)
pred_coords = scaler_coords.inverse_transform(pred_coords_scaled)
# --- [修正结束] ---

rmse = np.sqrt(np.mean((true_coords - pred_coords)**2))
ade = np.mean(np.sqrt(np.sum((true_coords - pred_coords)**2, axis=1)))

print("\n--- Keras TKAN 模型评估结果 ---")
print(f"Test Loss (Huber): {test_loss:.4f}")
print(f"Test MSE: {test_mse:.4f} (米^2, 标准化后)")
print("-" * 20)
print(f"总均方根误差 (RMSE): {rmse:.4f} (米)")
print(f"平均位移误差 (ADE/FDE): {ade:.4f} (米)")
print("---------------------------------")


# ==============================================================================
#  第五部分：保存最终预测结果
# ==============================================================================
print("\n--- 步骤 5: 保存预测结果到 CSV 文件 ---")
results_df = pd.DataFrame({
    'True_X': true_coords[:, 0],
    'True_Y': true_coords[:, 1],
    'Pred_X': pred_coords[:, 0],
    'Pred_Y': pred_coords[:, 1]
})

output_filename = 'results/tkan_UWP.csv'
os.makedirs('results', exist_ok=True) # 确保 results 文件夹存在
results_df.to_csv(output_filename, index=False)

print(f"✅ 预测结果已成功保存到: {output_filename}")



--- 步骤 0: 设备验证 ---
✅ PyTorch 检测到 CUDA GPU 可用。
  - GPU 型号: NVIDIA GeForce RTX 3070 Ti
✅ Keras 张量默认被创建在: cuda:0
✅ Keras 与 PyTorch 设备匹配。
结论：模型将运行在 'CUDA' 设备上。
成功加载数据集，总行数: 13824

Data loaded successfully!
Training data shape:   (11059, 22)
Validation data shape: (1382, 22)
Testing data shape:    (1383, 22)

Identified 13 WiFi features and 6 IMU features.
数据已在DataFrame上完成标准化。

--- 步骤 2: 创建滑动窗口数据 ---
滑动窗口数据创建完毕！
  - X_train shape: (11037, 20, 19)
  - y_train shape: (11037, 6)

--- 步骤 3: 构建与训练 Keras TKAN 模型 ---


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 20, 19)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ TKAN_Layer (TKAN)               │ (None, 32)             │         9,166 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Prediction_Head (Dense)         │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,364 (36.58 KB)

 Trainable params: 9,174 (35.84 KB)

 Non-trainable params: 190 (760.00 B)


--- 开始模型训练 (model.fit) ---
Epoch 1/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 47s 274ms/step - loss: 0.3064 - mse: 0.6703 - val_loss: 0.0529 - val_mse: 0.1058
Epoch 2/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 57s 332ms/step - loss: 0.0557 - mse: 0.1121 - val_loss: 0.0582 - val_mse: 0.1164
Epoch 3/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 49s 281ms/step - loss: 0.0262 - mse: 0.0524 - val_loss: 0.0536 - val_mse: 0.1072
Epoch 4/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 48s 278ms/step - loss: 0.0223 - mse: 0.0446 - val_loss: 0.0506 - val_mse: 0.1011
Epoch 5/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 51s 295ms/step - loss: 0.0199 - mse: 0.0399 - val_loss: 0.0479 - val_mse: 0.0958
Epoch 6/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 51s 297ms/step - loss: 0.0182 - mse: 0.0364 - val_loss: 0.0454 - val_mse: 0.0908
Epoch 7/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 51s 292ms/step - loss: 0.0168 - mse: 0.0337 - val_loss: 0.0430 - val_mse: 0.0860
Epoch 8/20
173/173 ━━━━━━━━━━━━━━━━━━━━ 53s 304ms/step - loss: 0.0156 - mse: 0.0313 - val_loss: 0.0406 - val_mse: 0.0812
Epoc

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import time  # <--- 【新增】导入 time 模块

# ==============================================================================
#  环境设置与依赖检查
# ==============================================================================

# 设置 Keras 后端为 PyTorch
os.environ['KERAS_BACKEND'] = 'torch'

try:
    import keras
    import torch 
    from packaging.version import parse as parse_version
    
    if parse_version(keras.__version__) < parse_version("3.0.0"):
        print("错误：检测到过时的 Keras 版本。需要 Keras 3+。")
        exit()

    from tkan import TKAN 

except ImportError as e:
    print(f"导入错误: {e}")
    exit()

# ==============================================================================
#  设备验证模块
# ==============================================================================
print("\n" + "="*80)
print("--- 步骤 0: 设备验证 ---")
device_type = "cpu"
if torch.cuda.is_available():
    device_type = "cuda"
    print(f"✅ PyTorch 检测到 CUDA GPU: {torch.cuda.get_device_name(0)}")
else:
    print("❌ 警告：未检测到 GPU，将在 CPU 上运行。")
print("="*80)

# ==============================================================================
#  第一部分：数据加载与预处理
# ==============================================================================
print("\n--- 步骤 1: 加载与预处理数据 ---")
# dictory = "../Datasets/"
# # 请确保文件名正确
# filename = "0.275_Speed_OB.csv" 

# # 为了防止路径错误，增加简单的异常处理
# try:
#     data = pd.read_csv(dictory+filename)
#     # data = pd.read_csv(r'../Datasets/zoulang/test.csv') # 如果需要替换测试数据
# except FileNotFoundError:
#     print(f"❌ 错误：找不到文件 {dictory+filename}")
#     print("请检查路径或文件名。")
#     exit()

# print(f"成功加载数据集，总行数: {len(data)}")

# # 数据分割
# train_ratio = 0.8
# val_ratio = 0.1
# data_size = len(data)
# train_end_idx = int(train_ratio * data_size)
# val_end_idx = train_end_idx + int(val_ratio * data_size)

# train_data = data.iloc[:train_end_idx]
# val_data = data.iloc[train_end_idx:val_end_idx]
# test_data = data.iloc[val_end_idx:]

train_data = pd.read_csv(
    r'../Datasets/zoulang/train.csv')
val_data = pd.read_csv(
    r'../Datasets/zoulang/validation.csv')
test_data = pd.read_csv(
    r'../Datasets/zoulang/test.csv')
# 特征列定义
column_names = train_data.drop(['timestamp','x_coord', 'y_coord'], axis=1, errors='ignore').columns.tolist()
wifi_features = [col for col in column_names if any(sensor in col for sensor in ["rot", "RSSI"])]
imu_features = [col for col in column_names if any(sensor in col for sensor in ["accelerometer", "gyroscope","magnetometer"])] 
coord_cols = ['x_coord', 'y_coord']

print(f"\nIdentified {len(wifi_features)} WiFi features and {len(imu_features)} IMU features.")

# 标准化
scaler_wifi = StandardScaler().fit(train_data[wifi_features])
scaler_imu = StandardScaler().fit(train_data[imu_features])
# 【关键】单独定义 label 的 scaler，方便后续反标准化
scaler_label = StandardScaler().fit(train_data[coord_cols]) 

train_df_scaled = train_data.copy()
val_df_scaled = val_data.copy()
test_df_scaled = test_data.copy()

for df in [train_df_scaled, val_df_scaled, test_df_scaled]:
    df[wifi_features] = scaler_wifi.transform(df[wifi_features])
    df[imu_features] = scaler_imu.transform(df[imu_features])
    df[coord_cols] = scaler_label.transform(df[coord_cols])

print("数据标准化完成。")

# ==============================================================================
#  第二部分：数据塑形 - 创建滑动窗口
# ==============================================================================
def create_sliding_windows(df, feature_cols, label_cols, window_size, future_steps):
    X, y = [], []
    num_samples = len(df) - window_size - future_steps + 1
    
    feature_data = df[feature_cols].values
    label_data = df[label_cols].values
    
    for i in range(num_samples):
        feature_window = feature_data[i : i + window_size]
        X.append(feature_window)
        # 展平 y 以适配 Dense 层输出
        label_window = label_data[i + window_size : i + window_size + future_steps]
        y.append(label_window.flatten())
        
    return np.array(X), np.array(y)

print("\n--- 步骤 2: 创建滑动窗口数据 ---")
windows_size = 30
future_steps = 3

all_features = wifi_features + imu_features
X_train, y_train = create_sliding_windows(train_df_scaled, all_features, coord_cols, windows_size, future_steps)
X_val, y_val = create_sliding_windows(val_df_scaled, all_features, coord_cols, windows_size, future_steps)
X_test, y_test = create_sliding_windows(test_df_scaled, all_features, coord_cols, windows_size, future_steps)

print(f"Test data shape: {X_test.shape}")

# ==============================================================================
#  第三部分：Keras TKAN 模型训练
# ==============================================================================
print("\n--- 步骤 3: 构建与训练 Keras TKAN 模型 ---")

input_shape = (X_train.shape[1], X_train.shape[2])
output_dim = y_train.shape[1]

inputs = keras.Input(shape=input_shape)
# TKAN 层
tkan_output = TKAN(units=32, return_sequences=False, name="TKAN_Layer")(inputs)
outputs = keras.layers.Dense(output_dim, name="Prediction_Head")(tkan_output)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.Huber(),
    metrics=['mse']
)

# 减少 epochs 以便测试代码，实际使用可改回 50
print("\n--- 开始模型训练 ---")
history = model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=50, # 如果只是测试代码，可以改小，比如 5
    validation_data=(X_val, y_val),
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
    verbose=2
)

# ==============================================================================
#  第四部分：评估与计时 (包含毫秒级统计)
# ==============================================================================
print("\n" + "="*80)
print("--- 步骤 4: 推理速度测试与评估 ---")
print("="*80)

# 1. 预热 (Warm-up)
# Keras/PyTorch 第一次运行由于图构建或内存分配，速度通常较慢。
print("正在进行模型预热...")
if len(X_test) > 0:
    # 取一小部分数据进行一次空跑
    _ = model.predict(X_test[:32], verbose=0)
print("预热完成。")

# 2. 正式计时
print("开始在测试集上进行全量预测...")

# 如果是 CUDA 环境，必须同步
if device_type == "cuda":
    torch.cuda.synchronize()

start_time = time.time() # ⏱️ 计时开始

# 执行预测 (verbose=0 防止打印进度条影响计时)
y_pred = model.predict(X_test, verbose=0)

if device_type == "cuda":
    torch.cuda.synchronize()

end_time = time.time()   # ⏱️ 计时结束

# 3. 计算时间指标
total_time_sec = end_time - start_time
total_samples = len(X_test)
avg_time_ms = (total_time_sec / total_samples) * 1000 # 转换为毫秒

print("\n" + "="*50)
print(f"INFERENCE SPEED REPORT (TKAN)")
print("-" * 50)
print(f"Device:               {device_type.upper()}")
print(f"Total Samples:        {total_samples}")
print(f"Total Time:           {total_time_sec:.4f} s")
print(f"FPS (Samples/Sec):    {total_samples/total_time_sec:.2f}")
print(f"Avg Time per Sample:  {avg_time_ms:.4f} ms (毫秒)") # <--- 您需要的单位
print("="*50 + "\n")


# 4. 计算误差指标
y_test_seq = y_test.reshape(-1, future_steps, 2)
y_pred_seq = y_pred.reshape(-1, future_steps, 2)

# 取最后一步进行评估
true_coords_scaled = y_test_seq[:, -1, :]
pred_coords_scaled = y_pred_seq[:, -1, :]

# 反标准化 (使用之前定义的 scaler_label)
# 注意：scaler_label 是针对 [x, y] 两列 fit 的
true_coords = scaler_label.inverse_transform(true_coords_scaled)
pred_coords = scaler_label.inverse_transform(pred_coords_scaled)

rmse = np.sqrt(np.mean(np.sum((true_coords - pred_coords)**2, axis=1))) # 修正RMSE计算方式 (先平方和再平均再开方)
# 或者: rmse = np.sqrt(np.mean((true_coords - pred_coords)**2)) 如果是想算 element-wise 的 RMSE

# ADE (Average Displacement Error) - 欧氏距离的平均值
displacement = np.sqrt(np.sum((true_coords - pred_coords)**2, axis=1))
ade = np.mean(displacement)

print(f"Test RMSE: {rmse:.4f} m")
print(f"Test ADE:  {ade:.4f} m")

# ==============================================================================
#  第五部分：保存结果
# ==============================================================================
print("\n--- 步骤 5: 保存预测结果 ---")
results_df = pd.DataFrame({
    'True_X': true_coords[:, 0],
    'True_Y': true_coords[:, 1],
    'Pred_X': pred_coords[:, 0],
    'Pred_Y': pred_coords[:, 1]
})

output_filename = 'results/tkan_zoulang.csv'
os.makedirs('results', exist_ok=True)
# results_df.to_csv(output_filename, index=False)
print(f"✅ 结果已保存至: {output_filename}")


--- 步骤 0: 设备验证 ---
✅ PyTorch 检测到 CUDA GPU: NVIDIA GeForce RTX 3070 Ti

--- 步骤 1: 加载与预处理数据 ---

Identified 6 WiFi features and 9 IMU features.
数据标准化完成。

--- 步骤 2: 创建滑动窗口数据 ---
Test data shape: (764, 30, 15)

--- 步骤 3: 构建与训练 Keras TKAN 模型 ---

--- 开始模型训练 ---
Epoch 1/50
12/12 - 4s - 370ms/step - loss: 0.4368 - mse: 0.9689 - val_loss: 0.4463 - val_mse: 0.9995
Epoch 2/50
12/12 - 5s - 387ms/step - loss: 0.4187 - mse: 0.9258 - val_loss: 0.4306 - val_mse: 0.9620
Epoch 3/50
12/12 - 5s - 390ms/step - loss: 0.4042 - mse: 0.8916 - val_loss: 0.4176 - val_mse: 0.9323
Epoch 4/50
12/12 - 5s - 376ms/step - loss: 0.3913 - mse: 0.8621 - val_loss: 0.4057 - val_mse: 0.9056
Epoch 5/50
12/12 - 5s - 380ms/step - loss: 0.3789 - mse: 0.8339 - val_loss: 0.3939 - val_mse: 0.8800
Epoch 6/50
12/12 - 5s - 381ms/step - loss: 0.3659 - mse: 0.8038 - val_loss: 0.3811 - val_mse: 0.8499
Epoch 7/50
12/12 - 5s - 383ms/step - loss: 0.3515 - mse: 0.7693 - val_loss: 0.3676 - val_mse: 0.8187
Epoch 8/50
12/12 - 5s - 388ms/step 